In [33]:
import pandas as pd

moves = pd.read_csv("data/moves.csv")
text = moves[["Name", "Effect"]]
combined_move = text.apply(lambda x: x["Name"] + ": " + x["Effect"], axis = 1).tolist()
text = ' '.join(str(elem) for elem in combined_move)

In [34]:
text

"Accelerock: The user smashes into the target at high speed. This move always goes first. Acrobatics: The user nimbly strikes the target. If the user is not holding an item, this attack inflicts massive damage. Aerial Ace: The user confounds the target with speed, then slashes. This attack never misses. Anchor Shot: The user entangles the target with its anchor chain while attacking. The target becomes unable to flee. Aqua Jet: The user lunges at the target at a speed that makes it almost invisible. This move always goes first. Aqua Tail: The user attacks by swinging its tail as if it were a vicious wave in a raging storm. Arm Thrust: The user lets loose a flurry of open-palmed arm thrusts that hit two to five times in a row. Assurance: If the target has already taken some damage in the same turn, this attack's power is doubled. Astonish: The user attacks the target while shouting in a startling fashion. This may also make the target flinch. Attack Order: The user calls out its underli

In [35]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io



#path = keras.utils.get_file(
#    "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
#)
#with io.open(path, encoding="utf-8") as f:
#    text = f.read().lower()

#code obtained from keras tutorial



#text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
#create dictionaries from chars to indices and indices to chars for ease of use

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters

#here, we cut sequences and move them over to create our training sequences

#I wanna know how long each move description is on average




Corpus length: 83596
Total chars: 71


In [36]:
np.mean(moves.Effect.apply(lambda x: len(x)))

99.29878869448183

In [44]:
maxlen = 50
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))


# I believe this creates the one hot vectors for the sentences, with respect to the chars

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
    

    
model = keras.Sequential(
# shape used to be maxlen
    [
        keras.Input(shape=(None, len(chars))),
        layers.LSTM(64),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)





def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


epochs = 40
batch_size = 128



Number of sequences: 27849


In [45]:
rnn_gen = model.fit(x, y, batch_size=batch_size, epochs = 30)

Epoch 1/30
218/218 [==============================] - 6s 22ms/step - loss: 2.9154
Epoch 2/30
218/218 [==============================] - 5s 22ms/step - loss: 1.9409
Epoch 3/30
218/218 [==============================] - 5s 22ms/step - loss: 1.6951
Epoch 4/30
218/218 [==============================] - 5s 22ms/step - loss: 1.5591
Epoch 5/30
218/218 [==============================] - 5s 22ms/step - loss: 1.4503
Epoch 6/30
218/218 [==============================] - 5s 22ms/step - loss: 1.3837
Epoch 7/30
218/218 [==============================] - 5s 22ms/step - loss: 1.3291
Epoch 8/30
218/218 [==============================] - 5s 22ms/step - loss: 1.2728
Epoch 9/30
218/218 [==============================] - 5s 22ms/step - loss: 1.2203
Epoch 10/30
218/218 [==============================] - 5s 23ms/step - loss: 1.1881
Epoch 11/30
218/218 [==============================] - 5s 22ms/step - loss: 1.1599
Epoch 12/30
218/218 [==============================] - 5s 22ms/step - loss: 1.1227
Epoch 13/30
2

In [48]:
def generate_move_description(sentence, model, diversity):
    
    generated = ""
    print('...Generating with seed: "' + sentence + '"')

    for i in range(100):
        x_pred = np.zeros((1, len(sentence), len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

    print("...Generated: ", generated)
    print()
    

for x in [0.1, 0.15, 0.2, 0.5, 0.7]:
    generate_move_description("Surging Strikes: This move ", model, x)

...Generating with seed: "Surging Strikes: This move "
...Generated:  enost by the target with a burn. This move enost by the target with a burn. This move enost by harsh

...Generating with seed: "Surging Strikes: This move "
...Generated:  enost by the target with a burn. This move entinges the target with a burn. Steep Shectoners its mov

...Generating with seed: "Surging Strikes: This move "
...Generated:  enost to its target. This move enost be used to it the target with a burn. This move enost by the ta

...Generating with seed: "Surging Strikes: This move "
...Generated:  entwifters the target with a burn. This move entarp stats the target. This move endites the target w

...Generating with seed: "Surging Strikes: This move "
...Generated:  entinges the target to inferes Elitting of a Puck the target in the target's the target on electrice



In [51]:
for x in [0.1, 0.15, 0.2, 0.5, 0.7]:
    generate_move_description("Accelerock:", model, x)
    
for x in [0.1, 0.15, 0.2, 0.5, 0.7]:
    generate_move_description("This move raises the Defense stat and causes ", model, x)

...Generating with seed: "Accelerock:"
...Generated:   The user's Sp. Atk status onthel the target. This move also ins becomes the target. This move to it

...Generating with seed: "Accelerock:"
...Generated:   The user's Sp. Atk stats its the target. This move to inflict move electriclessir Pokémon with a bu

...Generating with seed: "Accelerock:"
...Generated:   The user's Defense statical store the user quale hit by the target's and Sp. Atk status actack the 

...Generating with seed: "Accelerock:"
...Generated:   The user's Sp. Def stats its the target's HP. This move alwores force burbles the target's lowers t

...Generating with seed: "Accelerock:"
...Generated:   The user's croring itsision. The target's leaves the more easily Windhed with use throabies also st

...Generating with seed: "This move raises the Defense stat and causes "
...Generated:  it in a gripinges the target with a burn. This move enost by the target with a burn. This move entar

...Generating with seed: "This

In [50]:
# in order to use something like GPT-2 to generate domain specific text
# we'll need to fine tune a causal language model, and then use a sampling regime to generate the move text
# it would also be a good idea to include some additional information on the move somehow, to fine tune the output
